## Preprocessing

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.

application_df = application_df.drop(columns=["EIN", "NAME"])


In [4]:
# Determine the number of unique values in each column
unique_counts = application_df.nunique()

# Print the unique value counts
print(unique_counts)


APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [5]:
# Get value counts for the APPLICATION_TYPE column
application_type_counts = application_df["APPLICATION_TYPE"].value_counts()

# Print the value counts
print(application_type_counts)


T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


In [6]:
# Choose a cutoff value (e.g., 500) for the number of occurrences
cutoff_value = 500

# Create a list of application types to be replaced
application_types_to_replace = application_type_counts[application_type_counts < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, "Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()


T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Get value counts for the CLASSIFICATION column
classification_counts = application_df["CLASSIFICATION"].value_counts()

# Print the value counts
print(classification_counts)


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [8]:
# Get value counts for the CLASSIFICATION column where count is greater than 1
classification_counts_gt_1 = classification_counts[classification_counts > 1]

# Print the value counts
print(classification_counts_gt_1)


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64


In [9]:
# Choose a cutoff value (e.g., 1000) for the number of occurrences
cutoff_value = 1000

# Create a list of classifications to be replaced
classifications_to_replace = classification_counts[classification_counts < cutoff_value].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, "Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()


C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [10]:
# Convert categorical data to numeric using pd.get_dummies
application_df = pd.get_dummies(application_df, columns=["APPLICATION_TYPE", "AFFILIATION", "CLASSIFICATION", "USE_CASE", "ORGANIZATION", "INCOME_AMT", "SPECIAL_CONSIDERATIONS"])


In [11]:
# Split the data into features (X) and target (y)
X = application_df.drop(columns=["IS_SUCCESSFUL"])
y = application_df["IS_SUCCESSFUL"]

# Import the train_test_split function
from sklearn.model_selection import train_test_split

# Split the data into training and testing datasets (e.g., 80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)


In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [13]:
import tensorflow as tf

# Create a Sequential model
nn = tf.keras.models.Sequential()

# Define the number of input features
input_features = len(X_train_scaled[0])

# First hidden layer with, for example, 80 nodes and a ReLU activation function
nn.add(tf.keras.layers.Dense(units=80, input_dim=input_features, activation='relu'))

# Second hidden layer with, for example, 30 nodes and a ReLU activation function
nn.add(tf.keras.layers.Dense(units=30, activation='relu'))

# Output layer with a single neuron for binary classification (sigmoid activation)
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3520      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5981 (23.36 KB)
Trainable params: 5981 (23.36 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [15]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs=100, verbose=1)



Epoch 1/100
804/804 [==============================] - 1s 915us/step - loss: 0.5731 - accuracy: 0.7187
Epoch 2/100
804/804 [==============================] - 1s 864us/step - loss: 0.5562 - accuracy: 0.7289
Epoch 3/100
804/804 [==============================] - 1s 921us/step - loss: 0.5520 - accuracy: 0.7303
Epoch 4/100
804/804 [==============================] - 1s 850us/step - loss: 0.5512 - accuracy: 0.7314
Epoch 5/100
804/804 [==============================] - 1s 867us/step - loss: 0.5498 - accuracy: 0.7325
Epoch 6/100
804/804 [==============================] - 1s 877us/step - loss: 0.5483 - accuracy: 0.7329
Epoch 7/100
804/804 [==============================] - 1s 849us/step - loss: 0.5473 - accuracy: 0.7330
Epoch 8/100
804/804 [==============================] - 1s 866us/step - loss: 0.5472 - accuracy: 0.7332
Epoch 9/100
804/804 [==============================] - 1s 891us/step - loss: 0.5469 - accuracy: 0.7320
Epoch 10/100
804/804 [==============================] - 1s 874us/step - l

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


268/268 - 0s - loss: 0.5631 - accuracy: 0.7271 - 258ms/epoch - 961us/step
Loss: 0.5630907416343689, Accuracy: 0.7271137237548828


In [28]:
# Save the optimized model with a filename that contains only ASCII characters
model.save("model.h5")



In [18]:
import tensorflow as tf

# Define the model
nn_optimized = tf.keras.models.Sequential()

# Input layer
input_features = len(X_train_scaled[0])
nn_optimized.add(tf.keras.layers.Dense(units=80, input_dim=input_features, activation='relu'))

# Hidden layers
nn_optimized.add(tf.keras.layers.Dense(units=100, activation='relu'))
nn_optimized.add(tf.keras.layers.Dense(units=50, activation='relu'))

# Output layer
nn_optimized.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compile the model with a lower learning rate and different optimizer
nn_optimized.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=['accuracy'])

# Train the model with different epochs and batch size
history = nn_optimized.fit(X_train_scaled, y_train, epochs=50, batch_size=64, validation_data=(X_test_scaled, y_test), verbose=2)

# Evaluate the optimized model
model_loss, model_accuracy = nn_optimized.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


Epoch 1/50
402/402 - 1s - loss: 0.6244 - accuracy: 0.6840 - val_loss: 0.5810 - val_accuracy: 0.7177 - 1s/epoch - 3ms/step
Epoch 2/50
402/402 - 1s - loss: 0.5683 - accuracy: 0.7248 - val_loss: 0.5661 - val_accuracy: 0.7203 - 516ms/epoch - 1ms/step
Epoch 3/50
402/402 - 1s - loss: 0.5584 - accuracy: 0.7291 - val_loss: 0.5618 - val_accuracy: 0.7234 - 511ms/epoch - 1ms/step
Epoch 4/50
402/402 - 1s - loss: 0.5544 - accuracy: 0.7308 - val_loss: 0.5590 - val_accuracy: 0.7270 - 501ms/epoch - 1ms/step
Epoch 5/50
402/402 - 0s - loss: 0.5519 - accuracy: 0.7310 - val_loss: 0.5578 - val_accuracy: 0.7263 - 495ms/epoch - 1ms/step
Epoch 6/50
402/402 - 1s - loss: 0.5503 - accuracy: 0.7315 - val_loss: 0.5563 - val_accuracy: 0.7268 - 503ms/epoch - 1ms/step
Epoch 7/50
402/402 - 1s - loss: 0.5490 - accuracy: 0.7331 - val_loss: 0.5569 - val_accuracy: 0.7285 - 510ms/epoch - 1ms/step
Epoch 8/50
402/402 - 0s - loss: 0.5481 - accuracy: 0.7315 - val_loss: 0.5558 - val_accuracy: 0.7294 - 492ms/epoch - 1ms/step
Epo

In [19]:
import tensorflow as tf

# Create a Sequential model
nn_optimized = tf.keras.models.Sequential()

# Input layer
input_features = len(X_train_scaled[0])
nn_optimized.add(tf.keras.layers.Dense(units=100, input_dim=input_features, activation='relu'))

# Hidden layers with dropout regularization
nn_optimized.add(tf.keras.layers.Dropout(0.2))  # Dropout layer to prevent overfitting
nn_optimized.add(tf.keras.layers.Dense(units=50, activation='relu'))
nn_optimized.add(tf.keras.layers.Dropout(0.2))  # Dropout layer
nn_optimized.add(tf.keras.layers.Dense(units=25, activation='relu'))

# Output layer
nn_optimized.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compile the model with a lower learning rate and different optimizer
nn_optimized.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=['accuracy'])

# Train the model with different epochs and batch size
history = nn_optimized.fit(X_train_scaled, y_train, epochs=100, batch_size=128, validation_data=(X_test_scaled, y_test), verbose=2)

# Evaluate the optimized model
model_loss, model_accuracy = nn_optimized.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")




Epoch 1/100
201/201 - 1s - loss: 0.6965 - accuracy: 0.5768 - val_loss: 0.6297 - val_accuracy: 0.7022 - 1s/epoch - 5ms/step
Epoch 2/100
201/201 - 0s - loss: 0.6227 - accuracy: 0.6821 - val_loss: 0.5878 - val_accuracy: 0.7178 - 300ms/epoch - 1ms/step
Epoch 3/100
201/201 - 0s - loss: 0.5981 - accuracy: 0.7083 - val_loss: 0.5763 - val_accuracy: 0.7191 - 339ms/epoch - 2ms/step
Epoch 4/100
201/201 - 0s - loss: 0.5869 - accuracy: 0.7133 - val_loss: 0.5726 - val_accuracy: 0.7163 - 293ms/epoch - 1ms/step
Epoch 5/100
201/201 - 0s - loss: 0.5815 - accuracy: 0.7189 - val_loss: 0.5703 - val_accuracy: 0.7200 - 325ms/epoch - 2ms/step
Epoch 6/100
201/201 - 0s - loss: 0.5788 - accuracy: 0.7186 - val_loss: 0.5690 - val_accuracy: 0.7207 - 299ms/epoch - 1ms/step
Epoch 7/100
201/201 - 0s - loss: 0.5764 - accuracy: 0.7201 - val_loss: 0.5674 - val_accuracy: 0.7219 - 275ms/epoch - 1ms/step
Epoch 8/100
201/201 - 0s - loss: 0.5718 - accuracy: 0.7213 - val_loss: 0.5656 - val_accuracy: 0.7220 - 278ms/epoch - 1ms/

Epoch 66/100
201/201 - 0s - loss: 0.5510 - accuracy: 0.7321 - val_loss: 0.5541 - val_accuracy: 0.7292 - 290ms/epoch - 1ms/step
Epoch 67/100
201/201 - 0s - loss: 0.5499 - accuracy: 0.7321 - val_loss: 0.5539 - val_accuracy: 0.7293 - 283ms/epoch - 1ms/step
Epoch 68/100
201/201 - 0s - loss: 0.5509 - accuracy: 0.7335 - val_loss: 0.5537 - val_accuracy: 0.7292 - 287ms/epoch - 1ms/step
Epoch 69/100
201/201 - 0s - loss: 0.5501 - accuracy: 0.7322 - val_loss: 0.5539 - val_accuracy: 0.7294 - 278ms/epoch - 1ms/step
Epoch 70/100
201/201 - 0s - loss: 0.5509 - accuracy: 0.7304 - val_loss: 0.5538 - val_accuracy: 0.7296 - 283ms/epoch - 1ms/step
Epoch 71/100
201/201 - 0s - loss: 0.5498 - accuracy: 0.7304 - val_loss: 0.5536 - val_accuracy: 0.7294 - 285ms/epoch - 1ms/step
Epoch 72/100
201/201 - 0s - loss: 0.5495 - accuracy: 0.7308 - val_loss: 0.5535 - val_accuracy: 0.7296 - 276ms/epoch - 1ms/step
Epoch 73/100
201/201 - 0s - loss: 0.5501 - accuracy: 0.7328 - val_loss: 0.5535 - val_accuracy: 0.7294 - 302ms/e

In [30]:

# Define the file path for saving the model
model_filename = "optimized_model.h5"  # Specify the desired filename

# Save the model to an HDF5 file
nn.save(model_filename)

In [31]:
**Overview of the Analysis:**

The purpose of this analysis is to create a deep learning model to predict the success of organizations that receive funding from Alphabet Soup, a nonprofit foundation. The model aims to classify whether the money given to these organizations is used effectively (successful) or not. The goal is to optimize the model's accuracy above a specified threshold.

**Results:**

**Data Preprocessing:**

- **Target Variable:** The target variable for the model is "IS_SUCCESSFUL," which indicates whether the funding provided to an organization is used effectively (1) or not (0).

- **Features:** The following variables were used as features for the model:
  - APPLICATION_TYPE
  - AFFILIATION
  - CLASSIFICATION
  - USE_CASE
  - ORGANIZATION
  - STATUS
  - INCOME_AMT
  - SPECIAL_CONSIDERATIONS
  - ASK_AMT

- **Variables Removed:** The variables "EIN" (Employer Identification Number) and "NAME" were removed from the input data as they do not provide relevant information for the prediction task.

**Compiling, Training, and Evaluating the Model:**

- **Neurons, Layers, and Activation Functions:** The neural network model was designed with the following architecture:
  - Input layer: 100 neurons, ReLU activation
  - Hidden layer 1: 50 neurons, ReLU activation
  - Hidden layer 2: 25 neurons, ReLU activation
  - Output layer: 1 neuron, sigmoid activation (binary classification)

  These choices were made based on experimentation and the desire to create a model with enough complexity to capture patterns in the data while avoiding overfitting.

- **Achieving Target Performance:** The target model performance was to achieve an accuracy higher than 75%. The model was able to achieve an accuracy of approximately 72.92%, which, while improved from the initial model, fell slightly short of the target.

- **Steps to Increase Performance:** Several steps were taken to increase model performance, including data preprocessing (handling rare occurrences), adding hidden layers, adjusting the number of neurons, using ReLU activation functions, and training for more epochs. Despite these efforts, the target accuracy was not fully reached.

**Summary:**

The deep learning model showed promise in predicting the success of organizations funded by Alphabet Soup. While it achieved an improved accuracy of 72.92% after optimization, it fell slightly short of the 75% target. The model benefited from preprocessing steps, including binning rare occurrences and encoding categorical variables.

To further enhance model performance, additional strategies can be explored, such as fine-tuning hyperparameters, using different neural network architectures (e.g., convolutional or recurrent networks), and incorporating more advanced techniques like ensemble learning.

A different approach to solving this classification problem could involve trying different machine learning algorithms, such as Random Forest, Gradient Boosting, or Support Vector Machines. These algorithms may capture different patterns in the data and achieve higher accuracy levels. Additionally, feature engineering and a more extensive search for optimal hyperparameters could lead to improved results.

In conclusion, while the deep learning model shows promise, further exploration of different models and techniques is recommended to achieve the desired accuracy threshold for predicting the success of funded organizations.


SyntaxError: unterminated string literal (detected at line 3) (787773633.py, line 3)